In [2]:
from tracknet.model import GridTrackNetModel
from tracknet.dataset import TrackNet
from torchinfo import summary
import wandb

In [4]:
# wandb.login()

In [5]:
# wandb.init(project='GridTrackNet', name='wandb_setup_2')

model = GridTrackNetModel()

# wandb.watch(model, log_freq=500)

In [3]:
summary(model, input_size=(10, 15, 432, 768))

Layer (type:depth-idx)                   Output Shape              Param #
GridTrackNetModel                        [10, 15, 27, 48]          --
├─ConvBlock: 1-1                         [10, 64, 432, 768]        --
│    └─Sequential: 2-1                   [10, 64, 432, 768]        --
│    │    └─Conv2d: 3-1                  [10, 64, 432, 768]        8,704
│    │    └─ReLU: 3-2                    [10, 64, 432, 768]        --
│    │    └─BatchNorm2d: 3-3             [10, 64, 432, 768]        128
├─ConvBlock: 1-2                         [10, 64, 432, 768]        --
│    └─Sequential: 2-2                   [10, 64, 432, 768]        --
│    │    └─Conv2d: 3-4                  [10, 64, 432, 768]        36,928
│    │    └─ReLU: 3-5                    [10, 64, 432, 768]        --
│    │    └─BatchNorm2d: 3-6             [10, 64, 432, 768]        128
├─MaxPool2d: 1-3                         [10, 64, 216, 384]        --
├─ConvBlock: 1-4                         [10, 128, 216, 384]       --
│    └

In [6]:
import torch
import torch.nn as nn
from torch.optim import Adadelta
import numpy as np
from torch import cuda
from tqdm import tqdm

In [43]:
def validate(model, val_loader):
    device = "cuda" if cuda.is_available() else "cpu"
    model.to(device)
    model.eval()

    corrects = []

    for _, (instances, label) in enumerate(val_loader):
        with torch.no_grad():
            label = label.permute(0, 1, 4, 2, 3).reshape(val_loader.batch_size, 3 * 5, 27, 48)

            instances = instances.to(device, dtype=torch.float32)
            outputs = model(instances)

            for i in range(val_loader.batch_size):
                # Each 3 items is a grid confidence for 1 of 5 frames
                for j in range(0, 15, 3):
                    gt = np.argmax(label[i][j].flatten())
                    gt_x, gt_y = np.unravel_index(gt, (27, 48))

                    out = np.argmax(outputs[i][j].flatten())
                    out_x, out_y = np.unravel_index(out, (27, 48))

                    corrects.append(gt_x == out_x and gt_y == out_y)

            acc = sum(corrects) / len(corrects)

            print(acc)

            return
        
        # print(instances, label)

In [44]:
from torch.utils.data import DataLoader

val_dataset = TrackNet('compiled_dataset')
val_loader = DataLoader(val_dataset, batch_size=10)
validate(model, val_loader)

0.0


/var/folders/jb/63htsb4j5nx0ygnsyjz4jb_w0000gn/T/ipykernel_30745/205360393.py:19: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments. To learn more, see the migration guide https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword
  gt_x, gt_y = np.unravel_index(gt, (27, 48))
/var/folders/jb/63htsb4j5nx0ygnsyjz4jb_w0000gn/T/ipykernel_30745/205360393.py:22: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments. To learn more, see the migration guide https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword
  out_x, out_y = np.unravel_index(out, (27, 48))


In [ ]:

def train(model, train_loader, epochs = 10):
    device = "cuda" if cuda.is_available() else "cpu"
    model.to(device)
    model.train()

    criterion = nn.CrossEntropyLoss()
    optimizer = Adadelta(lr=1.0, params=model.parameters(), weight_decay=0.01)

    for epoch in range(epochs):
        losses = []

        for instances, label in tqdm(train_loader):
            optimizer.zero_grad()

            label = label.permute(0, 1, 4, 2, 3).reshape(train_loader.batch_size, 3 * 5, 27, 48)

            instances = instances.to(device, dtype=torch.float32)
            outputs = model(instances)

            print(outputs.shape, label.shape)
            
            loss = criterion(outputs, label)
            losses.append(loss)

            wandb.log({ 'train_loss': loss })

            loss.backward()
            optimizer.step()
        
        avg_loss = np.mean(losses)

        wandb.log({
            'epoch': epoch,
            'epoch_loss': avg_loss
        })


In [63]:
wandb.finish()

Error in callback <bound method _WandbInit._resume_backend of <wandb.sdk.wandb_init._WandbInit object at 0x16d7574d0>> (for pre_run_cell), with arguments args (<ExecutionInfo object at 161b3cde0, raw_cell="wandb.finish()" store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/Users/mrmorais/Code/grid-track-net/model_train.ipynb#X15sZmlsZQ%3D%3D>,),kwargs {}:


BrokenPipeError: [Errno 32] Broken pipe

BrokenPipeError: [Errno 32] Broken pipe

Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x16d7574d0>> (for post_run_cell), with arguments args (<ExecutionResult object at 161b3d8d0, execution_count=63 error_before_exec=None error_in_exec=[Errno 32] Broken pipe info=<ExecutionInfo object at 161b3cde0, raw_cell="wandb.finish()" store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/Users/mrmorais/Code/grid-track-net/model_train.ipynb#X15sZmlsZQ%3D%3D> result=None>,),kwargs {}:


BrokenPipeError: [Errno 32] Broken pipe

In [59]:
from torch.utils.data import DataLoader

train_dataset = TrackNet('compiled_dataset')
train_loader = DataLoader(train_dataset, batch_size=10)
train(model, train_loader)

  0%|          | 0/315 [00:00<?, ?it/s]

torch.Size([10, 15, 27, 48]) torch.Size([10, 15, 27, 48])


  0%|          | 1/315 [00:18<1:38:31, 18.83s/it]

torch.Size([10, 15, 27, 48]) torch.Size([10, 15, 27, 48])


  1%|          | 2/315 [00:36<1:33:09, 17.86s/it]

torch.Size([10, 15, 27, 48]) torch.Size([10, 15, 27, 48])


  1%|          | 2/315 [00:44<1:55:06, 22.07s/it]


KeyboardInterrupt: 

Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x16d7574d0>> (for post_run_cell), with arguments args (<ExecutionResult object at 2b079c2f0, execution_count=59 error_before_exec=None error_in_exec= info=<ExecutionInfo object at 2b079d240, raw_cell="from torch.utils.data import DataLoader

train_dat.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/Users/mrmorais/Code/grid-track-net/model_train.ipynb#W4sZmlsZQ%3D%3D> result=None>,),kwargs {}:


BrokenPipeError: [Errno 32] Broken pipe

In [14]:
import torch